<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-experiment/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 36.5 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
11.8


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu118
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone -b sg-experiment https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 110 (delta 53), reused 50 (delta 14), pack-reused 0
Receiving objects: 100% (110/110), 85.51 KiB | 3.89 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [5]:
!pip install torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.1 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=86cc96d2946d70a655925266f763bf233a8392fc3e671ef39b3f209206422d8c
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires pyth

In [6]:
import ssl

from graphwavenet import GraphWaveNet
ssl._create_default_https_context = ssl._create_unverified_context



# # look for other datasets
from torch_geometric_temporal.dataset import METRLADatasetLoader
loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
iterator = iter(dataset)
print("METRLADatasetLoader", next(iterator))
print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset()
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))

# from torch_geometric_temporal.dataset import PedalMeDatasetLoader
# dataset = PedalMeDatasetLoader().get_dataset()
# print("PedalMeDatasetLoader", next(iter(dataset)))


from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
iterator = iter(dataset)
print("WikiMathsDatasetLoader", next(iterator))
print("WikiMathsDatasetLoader", next(iterator))

# from torch_geometric_temporal.dataset import WindmillOutputSmallDatasetLoader
# dataset = WindmillOutputSmallDatasetLoader().get_dataset()
# print("WindmillOutputSmallDatasetLoader", next(iter(dataset)))

# from torch_geometric_temporal.dataset import EnglandCovidDatasetLoader
# dataset = EnglandCovidDatasetLoader().get_dataset()
# print("EnglandCovidDatasetLoader", next(iter(dataset)))


# from torch_geometric_temporal.dataset import MontevideoBusDatasetLoader
# dataset = MontevideoBusDatasetLoader().get_dataset()
# print("MontevideoBusDatasetLoader", next(iter(dataset)))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))

METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])
WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])


In [ ]:
# Run WikiMaths Web Traffic Dataset

from torch_geometric_temporal.signal import temporal_signal_split

# sample training loop
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)


import torch
import torch.nn.functional as F

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

in_dim = 14
out_dim = 1
num_nodes = 1068
timesteps_to_predict = 1
epochs = 50
lrate = 0.0005
wdecay = 0.0001
save_path = "store/checkpoint"

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

model.train()

for epoch in tqdm(range(epochs)):
    train_loss = []
    train_mape = []
    train_rmse = []
    t1 = time.time()

    for iter, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(1, num_nodes, in_dim).to(device)
        y_train = snapshot.y.to(device)
        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mae(pred, y_train, 0.0) # mean absolute error as loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        mape = masked_mape(pred, y_train, 0.0).item()
        rmse = masked_rmse(pred, y_train, 0.0).item()
        metrics = loss.item(), mape, rmse
        train_loss.append(metrics[0])
        train_mape.append(metrics[1])
        train_rmse.append(metrics[2])
        if iter % 100 == 0:
            log = 'Iter: {:03d}, Train Loss: {:.4f}, Train MAPE: ' + \
                '{:.4f}, Train RMSE: {:.4f}'
            print(
                log.format(iter, train_loss[-1], train_mape[-1],
                            train_rmse[-1]), flush=True)



# eval
model.eval()
loss = 0
for iter, snapshot in enumerate(test_dataset):
    x_test = snapshot.x.reshape(1, num_nodes, in_dim).to(device)
    y_test = snapshot.y.to(device)
    pred = model(x_test, snapshot.edge_index, snapshot.edge_attr)
    loss += masked_mae(pred, y_test, 0.0) # mean absolute error as loss
loss = loss / (iter+1)
loss = loss.item()
print("MAE: {:.4f}".format(loss))


start training...


  0%|          | 0/50 [00:00<?, ?it/s]

Iter: 000, Train Loss: 0.6313, Train MAPE: -0.4940, Train RMSE: 0.7430
Iter: 100, Train Loss: 0.4022, Train MAPE: 0.6207, Train RMSE: 0.6182
Iter: 200, Train Loss: 0.4201, Train MAPE: 0.4507, Train RMSE: 0.7092
Iter: 300, Train Loss: 0.3731, Train MAPE: 1.3640, Train RMSE: 0.6678


  2%|▏         | 1/50 [00:26<21:16, 26.04s/it]

Iter: 000, Train Loss: 0.4242, Train MAPE: -0.5135, Train RMSE: 0.5376
Iter: 100, Train Loss: 0.3600, Train MAPE: 0.6902, Train RMSE: 0.5820
Iter: 200, Train Loss: 0.4136, Train MAPE: 0.4420, Train RMSE: 0.7080
Iter: 300, Train Loss: 0.3871, Train MAPE: 3.3901, Train RMSE: 0.7051


...

Iter: 000, Train Loss: 0.3040, Train MAPE: -0.1930, Train RMSE: 0.4281
Iter: 100, Train Loss: 0.3654, Train MAPE: 0.2322, Train RMSE: 0.5905
Iter: 200, Train Loss: 0.4223, Train MAPE: 0.3305, Train RMSE: 0.7023
Iter: 300, Train Loss: 0.3334, Train MAPE: 2.6971, Train RMSE: 0.6397


 12%|█▏        | 6/50 [02:32<18:32, 25.28s/it]

Iter: 000, Train Loss: 0.3211, Train MAPE: -0.1796, Train RMSE: 0.4419
Iter: 100, Train Loss: 0.3692, Train MAPE: 0.3832, Train RMSE: 0.5899
Iter: 200, Train Loss: 0.4179, Train MAPE: 0.4000, Train RMSE: 0.7022
Iter: 300, Train Loss: 0.3282, Train MAPE: 1.7195, Train RMSE: 0.6263


 14%|█▍        | 7/50 [02:57<18:02, 25.17s/it]

Iter: 000, Train Loss: 0.3537, Train MAPE: -0.2202, Train RMSE: 0.4655
Iter: 100, Train Loss: 0.3717, Train MAPE: 0.3458, Train RMSE: 0.5930
Iter: 200, Train Loss: 0.4275, Train MAPE: 0.4795, Train RMSE: 0.7071
